In [1]:
!pip install nltk
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
!pip install tensorflow


In [2]:
from google.colab import files
uploaded = files.upload()   # will open a file picker


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [5]:
import pandas as pd
data = pd.read_csv("IMDB Dataset.csv")


In [6]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


In [7]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Load dataset
data = pd.read_csv("IMDB Dataset.csv")

# Preprocessing
english_stops = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r"<.*?>", "", text)        # remove HTML tags
    text = re.sub(r"[^A-Za-z]", " ", text)  # remove non-alphabet chars
    words = text.split()
    words = [w.lower() for w in words if w not in english_stops]
    return " ".join(words)

data["review"] = data["review"].apply(clean_text)
data["sentiment"] = data["sentiment"].map({"positive": 1, "negative": 0})

# Features & Labels
X = data["review"]
y = data["sentiment"]

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Padding
max_length = int(np.mean([len(x) for x in x_train_seq]))  # average length
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding="post")
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding="post")

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Build LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_length))
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print(model.summary())

# Save best model
checkpoint = ModelCheckpoint("best_lstm.h5", monitor="val_accuracy", save_best_only=True, verbose=1)

# Train
history = model.fit(
    x_train_pad, y_train,
    validation_split=0.2,
    batch_size=128,
    epochs=5,
    callbacks=[checkpoint],
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(x_test_pad, y_test, verbose=0)
print(f"\nTest Accuracy: {acc*100:.2f}%")

# Predict on new review
sample_review = "The movie was absolutely fantastic! Great acting and visuals."
cleaned = clean_text(sample_review)
seq = tokenizer.texts_to_sequences([cleaned])
padded = pad_sequences(seq, maxlen=max_length, padding="post")
prediction = model.predict(padded)

print("Prediction:", prediction)
print("Sentiment:", "Positive" if prediction >= 0.5 else "Negative")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

None
Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - accuracy: 0.5568 - loss: 0.6756
Epoch 1: val_accuracy improved from -inf to 0.66575, saving model to best_lstm.h5


250/250 ━━━━━━━━━━━━━━━━━━━━ 163s 638ms/step - accuracy: 0.5570 - loss: 0.6754 - val_accuracy: 0.6658 - val_loss: 0.6387
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - accuracy: 0.6872 - loss: 0.6033
Epoch 2: val_accuracy improved from 0.66575 to 0.75750, saving model to best_lstm.h5


250/250 ━━━━━━━━━━━━━━━━━━━━ 201s 636ms/step - accuracy: 0.6873 - loss: 0.6032 - val_accuracy: 0.7575 - val_loss: 0.5271
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 589ms/step - accuracy: 0.7503 - loss: 0.5426
Epoch 3: val_accuracy did not improve from 0.75750
250/250 ━━━━━━━━━━━━━━━━━━━━ 158s 630ms/step - accuracy: 0.7503 - loss: 0.5426 - val_accuracy: 0.7204 - val_loss: 0.5623
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.7319 - loss: 0.5408
Epoch 4: val_accuracy improved from 0.75750 to 0.81063, saving model to best_lstm.h5


250/250 ━━━━━━━━━━━━━━━━━━━━ 203s 634ms/step - accuracy: 0.7320 - loss: 0.5407 - val_accuracy: 0.8106 - val_loss: 0.4538
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.8534 - loss: 0.3784
Epoch 5: val_accuracy improved from 0.81063 to 0.85513, saving model to best_lstm.h5


250/250 ━━━━━━━━━━━━━━━━━━━━ 202s 634ms/step - accuracy: 0.8534 - loss: 0.3784 - val_accuracy: 0.8551 - val_loss: 0.3901

Test Accuracy: 85.55%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
Prediction: [[0.7861147]]
Sentiment: Positive
